In [ ]:
%pip install -q -r requirements.txt

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# load hyperparams instance
from hyper_params import HyperParams
params = HyperParams()

In [3]:
DATA_DIR='data'

In [4]:
from dataproc import io
io.DATA_DIR='data'

In [ ]:
# build all features dataset
import featues_datasets_all_patients
featues_datasets_all_patients.DATA_DIR = DATA_DIR
df_final_dataset = featues_datasets_all_patients.run(params)

In [6]:
df_final_dataset = io.load_dataframe('df_final_dataset')

In [7]:
# create a cohort dataframe and numpy file
import build_cohort_dataset
df_cohort = build_cohort_dataset.build_cohort(params, df_final_dataset)

df_labels: (4130, 2)
cohort dataset: (4085, 843)
cohort data saved to data/fulldata.npy


## Tools to explore too many unique values

In [ ]:
df_dataset_unprocessed[categorical_cols].nunique().sort_values(ascending=False)[0:20]

In [ ]:
allvals = df_dataset_unprocessed['51508'].unique().tolist()
len(allvals), df_dataset_unprocessed['51508'].notna().sum()

In [ ]:
df_dataset_unprocessed['51508'].value_counts()

In [ ]:
[x for x in allvals if x and not x == np.nan and not str(x).replace('.','').isnumeric()]

In [ ]:
pd.Series(sorted([float(x) for x in allvals if x and str(x).replace('.','').isnumeric() and float(x) > 1000.0])).describe()


In [ ]:
df_dataset_unprocessed.nunique().sort_values(ascending=False)

## THE END

In [ ]:
import torch
torch.__version__

In [ ]:
fulldata